# DeepXDE

In [ ]:
%%capture
!pip install pyfoil
!pip install deepxde
!pip install scikit-optimize
!pip3 install torch torchvision torchaudio
# !pip install tensorflow
# !export DDEBACKEND='tensorflow.compat.v1'

## Flow around profile



In [ ]:
import  numpy as np, types, skopt, os, pyfoil ,  pandas  as pd, matplotlib.pyplot as plt, json
  

from pathlib import Path 
import torch 
from deepxde.backend import torch 
import deepxde as dde 
dde.backend.load_backend('pytorch') 

from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import matplotlib.path as mpath
   
from PIL import Image 

In [ ]:
global rho; rho  = 1.0
global mu;  mu   = 0.02
global T;   T = 273 + 20
global R;   R = 8.341
from pde import *

###  Compressible (pattern)

In [ ]:
# Определение граничных областей
def boundary_farfield_inlet(x, on_boundary):
    return on_boundary and np.isclose(x[0], xmin)

def boundary_farfield_top_bottom(x, on_boundary):
    return on_boundary and (np.isclose(x[1], ymax) or np.isclose(x[1], ymin))

def boundary_farfield_outlet(x, on_boundary):
    return on_boundary and np.isclose(x[0], xmax)

def boundary_airfoil(x, on_boundary):
    if x.shape == (3,) :
      return on_boundary and point_on_contour(
          contour_points=foil_points,point=x[:-1],threshold=1e-5
          )


if dde.backend.backend_name=='tensorflow.compat.v1':
  class Polygon_v2(dde.geometry.Polygon):
      def __init__(self, vertices):
          super().__init__(vertices)
      @tf.function
      def distance2boundary_tf(self, x):
          distances = tf.norm(tf.constant(self.vertices, dtype=tf.float32) - tf.cast(x[:, tf.newaxis, :], tf.float32), axis=2)
          return tf.reduce_min(distances, axis=1)

      def distance2boundary_np(self, x):
          distances = []
          for point in x:
              min_distance = np.min(np.linalg.norm(self.vertices - point, axis=1))
              distances.append(min_distance)
          return np.array(distances)

      def distance2boundary(self, x):
          if isinstance(x, tf.Tensor):
              return self.distance2boundary_tf(x)
          elif isinstance(x, np.ndarray):
              return self.distance2boundary_np(x)
          else:
              raise ValueError("Input must be either a TensorFlow tensor or a NumPy array")

elif dde.backend.backend_name=='pytorch':
  class Polygon_v2(dde.geometry.Polygon):
      def __init__(self, vertices):
          super().__init__(vertices)
      def distance2boundary_torch(self, x):
          distances = torch.norm(torch.tensor(self.vertices, dtype=torch.float32) - torch.tensor(x[:, None, :], dtype=torch.float32), dim=2)
          return torch.min(distances, dim=1).value

      def distance2boundary_np(self, x):
          distances = []
          for point in x:
              min_distance = np.min(np.linalg.norm(self.vertices - point, axis=1))
              distances.append(min_distance)
          return np.array(distances)

      def distance2boundary(self, x):
          if isinstance(x, torch.Tensor):
              return self.distance2boundary_torch(x)
          elif isinstance(x, np.ndarray):
              return self.distance2boundary_np(x)
          else:
              raise ValueError("Input must be either a TensorFlow tensor or a NumPy array")

def compute_normals_tf(x, contour_points):
    dists = tf.norm(x[:, tf.newaxis, :] - contour_points, axis=2)
    closest_indices = tf.argmin(dists, axis=1)
    closest_contour_points = tf.gather(contour_points, closest_indices, axis=0)
    vectors = closest_contour_points - x
    normals = vectors / tf.norm(vectors, axis=1, keepdims=True)
    normals = tf.cast(normals, tf.float32)
    return normals

def compute_normals_torch(x, contour_points):
    x = torch.tensor(x, dtype=torch.float32)
    contour_points = torch.tensor(contour_points, dtype=torch.float32)

    dists = torch.norm(x.unsqueeze(1) - contour_points, dim=2)
    closest_indices = torch.argmin(dists, dim=1)
    closest_contour_points = torch.index_select(contour_points, 0, closest_indices)
    vectors = closest_contour_points - x
    normals = vectors / torch.norm(vectors, dim=1, keepdim=True)
    normals = normals.float()
    return normals

def compute_normals_np(x, contour_points):
    dists = np.linalg.norm(x[:, np.newaxis, :] - contour_points, axis=2)
    closest_indices = np.argmin(dists, axis=1)
    closest_contour_points = contour_points[closest_indices]
    vectors = closest_contour_points - x
    normals = vectors / np.linalg.norm(vectors, axis=1, keepdims=True)
    return normals

Начальные условия

#### Train

In [ ]:
# From file
numInternal         = 10000
numXmax             = 2500
numXmin             = 2500
numYmax             = 2500
numYmin             = 2500
# Make domain
inner_points_N      = 10000
outer_points_N      = 10000
farfield_points_N   = 2500
airfoil_points_N    = 2500

In [ ]:
# Чтение данных из CSV файла
df = pd.read_csv(os.path.join(os.getcwd(),'NACA0012', 'case_30000.csv'))
# Open the JSON file in read mode
with open(os.path.join(os.getcwd(),'NACA0012', "caseInfo.json"), 'r') as file:
    # Load the JSON data from the file into a Python object
    infoCase = json.load(file)

xmin, ymin, xmax, ymax, UMax, VMax, pMax = [infoCase[name] for name in ['xmin', 'ymin', 'xmax', 'ymax', 'UMax', 'VMax', 'pMax']]
# Извлечение значений из DataFrame
U_selected = df['U'].values.reshape(-1, 1)
V_selected = df['V'].values.reshape(-1, 1)
P_selected = df['P'].values.reshape(-1, 1)
x_selected = df['x'].values.reshape(-1, 1)
y_selected = df['y'].values.reshape(-1, 1)
# pMax = df['P'].max()
timeSteps = df['Time'].values.reshape(-1, 1)

xmin,xmax = x_selected.min(),x_selected.max(),
ymin,ymax = y_selected.min(),y_selected.max(),

In [ ]:
dde.config.set_random_seed(48)
dde.config.set_default_float('float32')

nacaParams = {
    'naca_code' : '0012',
    'c'         : 1,
    'offset_x'  : 0.0,
    'offset_y'  : 0.0,
    'angle_deg' : 0
}

from utils import *
foil_points = boundaryNACA(**nacaParams)

xmin_rec, xmax_rec = xmin + (-xmin + xmax)*1/4, xmin + (-xmin + xmax)*3/4
ymin_rec, ymax_rec = ymin + (-ymin + ymax)*1/4, ymin + (-ymin + ymax)*3/4

umax = UMax
global Re;  Re = rho * umax * nacaParams['c'] * 1 / mu

In [ ]:
print(f'xmin, xmax:{xmin, xmax}')
print(f'ymin, ymax:{ymin, ymax}')
print(f'xmin_rec, xmax_rec :{xmin_rec, xmax_rec }')
print(f'ymin_rec, ymax_rec:{ymin_rec, ymax_rec}')
print(f'rho:{rho}')
print(f'mu:{mu}')
print(f'umax:{umax}')
print(f'Re:{Re}')

In [ ]:
farfield    = dde.geometry.Rectangle([xmin, ymin], [xmax, ymax])
airfoil     = Polygon_v2(foil_points) # dde.geometry.Polygon(foil_points)

geom        = dde.geometry.CSGDifference(farfield, airfoil)
timedomain  = dde.geometry.TimeDomain(np.min(timeSteps), np.max(timeSteps))


inner_rec   = dde.geometry.Rectangle([xmin_rec, ymin_rec], [xmax_rec, ymax_rec])
outer_dom   = dde.geometry.CSGDifference(farfield, inner_rec)
outer_dom   = dde.geometry.CSGDifference(outer_dom, airfoil)
inner_dom   = dde.geometry.CSGDifference(inner_rec, airfoil)


def distance2boundary(self,x):
    return self.geometry.distance2boundary(x[:,:-1])


inner_dom_XTime = dde.geometry.GeometryXTime(inner_dom, timedomain)
outer_dom_XTime = dde.geometry.GeometryXTime(outer_dom, timedomain)
farfield_XTime  = dde.geometry.GeometryXTime(farfield, timedomain)

airfoil_XTime   = dde.geometry.GeometryXTime(airfoil, timedomain)


airfoil_XTime.distance2boundary = types.MethodType(distance2boundary, airfoil_XTime)

inner_points    = inner_dom_XTime.random_points(inner_points_N)
outer_points    = outer_dom_XTime.random_points(outer_points_N)
farfield_points = farfield_XTime.random_boundary_points(farfield_points_N)
airfoil_points  = airfoil_XTime.random_boundary_points(airfoil_points_N)

ob_xyt = np.hstack((x_selected, y_selected, timeSteps))
print(ob_xyt.shape)

points = np.concatenate((
    inner_points,
    outer_points,
    farfield_points,
    airfoil_points,
    ob_xyt
    ),axis=0)

geomtime = dde.geometry.GeometryXTime(geom, timedomain)

In [ ]:
# Placeholder для точек
contour_points = airfoil_XTime.uniform_boundary_points(500)
if dde.backend.backend_name=='tensorflow.compat.v1':contour_points_tf = tf.convert_to_tensor(contour_points, dtype=tf.float32)
if dde.backend.backend_name=='pytorch': contour_points_torch = torch.tensor(contour_points, dtype=torch.float32)

In [ ]:
## U # м^(1)*с^(-2)
def fun_U_farfield(x, y, _):return y[:,0:1] - UMax
def fun_no_slip_U(x, y, _): return y[:,0:1]
# V
def fun_V_farfield(x, y, _): return  y[:,1:2] - 0
def fun_no_slip_V(x, y, _): return y[:,1:2]

## U # м^(1)*с^(-2)
bc_inlet_u = dde.icbc.OperatorBC(geomtime, fun_no_slip_U, boundary_farfield_inlet)
bc_outlet_u = dde.icbc.OperatorBC(geomtime, fun_U_farfield, boundary_farfield_outlet)
bc_top_bottom_u = dde.icbc.OperatorBC(geomtime, fun_no_slip_U, boundary_farfield_top_bottom)
bc_airfoil_u = dde.icbc.OperatorBC(geomtime, fun_no_slip_U, boundary_airfoil)

bc_U = [bc_inlet_u, bc_outlet_u, bc_top_bottom_u, bc_airfoil_u]
observe_U = dde.icbc.PointSetBC(ob_xyt, U_selected, component=0)

## V # м^(1)*с^(-2)
bc_airfoil_v = dde.OperatorBC(geomtime, fun_no_slip_V, boundary_airfoil)
bc_inlet_v = dde.OperatorBC(geomtime, fun_no_slip_V, boundary_farfield_inlet)
bc_outlet_v = dde.OperatorBC(geomtime,fun_V_farfield, boundary_farfield_outlet)
bc_top_bottom_v = dde.OperatorBC(geomtime, fun_no_slip_V, boundary_farfield_top_bottom)

bc_V = [bc_inlet_v,bc_outlet_v,bc_top_bottom_v,bc_airfoil_v]
observe_V = dde.icbc.PointSetBC(ob_xyt, V_selected, component=1)

In [ ]:
pout = 0 # м^(2)*с^(-2)
internalField_p_value = pout

def internalField_p(x,y,_):
  if dde.backend.backend_name=='tensorflow.compat.v1':
    return y[:,4:5] - tf.cast(internalField_p_value, tf.float32)
  elif dde.backend.backend_name=='pytorch':
    return y[:,4:5] - torch.tensor(internalField_p_value, dtype=torch.float32)

def zeroGradient_p(x,y,_):
  p = y[:,4:5]
  return dde.grad.jacobian(p, x, j  = 0) \
  + dde.grad.jacobian(p, x,  j  = 1)

bc_inlet_p = dde.icbc.OperatorBC(
    geomtime, internalField_p, boundary_farfield_inlet, #component=5
    )
bc_outlet_p = dde.icbc.OperatorBC(
    geomtime, internalField_p, boundary_farfield_outlet, #component=5
    )
bc_top_bottom_p = dde.icbc.OperatorBC(
    geomtime, internalField_p, boundary_farfield_top_bottom, #component=5
    )
bc_airfoil_p = dde.icbc.OperatorBC(
    geomtime, zeroGradient_p, boundary_airfoil,#component = 5
    )

bc_p = [bc_inlet_p,bc_outlet_p,bc_top_bottom_p,bc_airfoil_p]

observe_p = dde.icbc.PointSetBC(ob_xyt, P_selected, component=2)

In [ ]:
# Problem setup
data = dde.data.TimePDE(
    geomtime,
    navier_stokes,
    [
        *bc_U,        observe_U, # 5
        *bc_V,        observe_V, # 10
        *bc_p,        observe_p, # 35
        ],
    num_domain = 0, num_boundary = 0,
    num_test = 5000,
    anchors = points,
)

In [ ]:
plt.figure(figsize = (32, 18))
plt.scatter(data.train_x_all[:,0], data.train_x_all[:,1], s = 0.3)
plt.scatter(ob_xyt[:,0],ob_xyt[:,1], s = 0.5, color='red')
plt.axis('equal')

In [ ]:
# Neural network definition
layer_size  = [3] + [50] * 10 + [6]
activation  = 'tanh'
initializer = 'Glorot uniform'

# FNN
net = dde.nn.FNN(layer_size, activation, initializer)

model_path = os.path.join(os.getcwd(),'model')
if not os.path.exists(model_path):
    os.makedirs(model_path)
    print(f"Папка '{model_path}' создана успешно.")
else:
    print(f"Папка '{model_path}' уже существует.")


# Model definition
model = dde.Model(data, net)
model.compile(optimizer = 'adam', lr = 5e-4)

checker = dde.callbacks.ModelCheckpoint(
    os.path.join(model_path,'model.ckpt'),
    save_better_only=True,
    period=5000
)


train_story_path = os.path.join(os.getcwd(),'train_story')
if not os.path.exists(train_story_path):
    os.makedirs(train_story_path)
    print(f"Папка '{train_story_path}' создана успешно.")
else:
    print(f"Папка '{train_story_path}' уже существует.")

losshistory, train_state = model.train(
    epochs = 250_000, display_every = 5000,
    model_save_path = os.path.join(train_story_path,'train_story'),
    callbacks=[checker]
    )
dde.saveplot(losshistory, train_state, issave = True, isplot = True)


##### Additional

In [ ]:
# To restore a model, use Model.restore.
model.restore("model/model.ckpt-?", verbose=1)  # Replace ? with the exact filename

In [ ]:
model.compile(optimizer = 'L-BFGS-B', loss_weights = [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2])
model.train_step.optimizer_kwargs = {'options': {'maxcor': 50,
                                                   'ftol': 1.0 * np.finfo(float).eps,
                                                   'maxfun':  2000,
                                                   'maxiter': 2000,
                                                   'maxls': 50}}
checker_bfgs = dde.callbacks.ModelCheckpoint(
    os.path.join(os.getcwd(),'model','model.ckpt'),
    save_better_only=True,
    period=1000
)

losshistory, train_state = model.train(display_every = 100, model_save_path = './')
dde.saveplot(losshistory, train_state, issave = True, isplot = True)

In [ ]:

# Plotting tool: thanks to @q769855234 code snippet
dx = 0.01
dy = 0.01
x = np.arange(xmin, xmax + dy, dx)
y = np.arange(ymin, ymax + dy, dy)

X = np.zeros((len(x)*len(y), 2))
xs = np.vstack((x,)*len(y)).reshape(-1)
ys = np.vstack((y,)*len(x)).T.reshape(-1)
X[:, 0] = xs
X[:, 1] = ys

def getU(x, y):
     return y[:, 0:1]

def getV(x, y):
     return y[:, 1:2]

def getP(x, y):
    return y[:, 2:3]

# Model predictions generation
u = model.predict(X, operator = getU)
v = model.predict(X, operator = getV)
p = model.predict(X, operator = getP)

for i in range(len(X)):
   if airfoil.inside(np.array([X[i]]))[0]:
       u[i] = 0.0
       v[i] = 0.0

u = u.reshape(len(y), len(x))
v = v.reshape(len(y), len(x))
p = p.reshape(len(y), len(x))

airfoil_plot = boundaryNACA4D(0, 0, 12, 0.2, 150, offset_x = 0.20, offset_y = 0.35)

fig1, ax1 = plt.subplots(figsize = (16, 9))
ax1.streamplot(x, y, u, v, density = 1.5)
clev = np.arange(p.min(), 2, 0.001)
cnt1 = ax1.contourf(x, y, p, clev, cmap = plt.cm.coolwarm, extend='both')
plt.axis('equal')
plt.fill(airfoil_plot[:, 0], airfoil_plot[:, 1])
fig1.colorbar(cnt1)
plt.savefig('NACA0012NS0.png')

fig2, ax2 = plt.subplots(figsize = (16, 9))
ax2.streamplot(x, y, u, v, density = 1.5)
clev = np.arange(0, u.max(), 0.001)
cnt2 = ax2.contourf(x, y, u, clev, cmap = plt.cm.coolwarm, extend='both')
plt.axis('equal')
plt.fill(airfoil_plot[:, 0], airfoil_plot[:, 1])
fig2.colorbar(cnt2)
plt.savefig('NACA0012NS1.png')

fig3, ax3 = plt.subplots(figsize = (16, 9))
ax3.streamplot(x, y, u, v, density = 1.5)
clev = np.arange(-0.235, v.max(), 0.001)
cnt3 = ax3.contourf(x, y, v, clev, cmap = plt.cm.coolwarm, extend='both')
plt.axis('equal')
plt.fill(airfoil_plot[:, 0], airfoil_plot[:, 1])
fig3.colorbar(cnt3)
plt.savefig('NACA0012NS2.png')

In [ ]:
!zip -r /./incompressible.zip /.

####  Predict


In [ ]:
!mkdir /content/incompressible_model
!tar -xvf "/content/incompressible_model.tar" -C "/content/incompressible_model"     #[run this cell to extract tar files]

In [ ]:
folder_path = os.path.join(
    os.getcwd(),
    'incompressible_model',
    'model'
)

files = os.listdir(folder_path)
max_index = -1
max_file = None

for file_name in files:
    if file_name.startswith('model.ckpt-'):
        index_str = file_name.split('-')[-1].split('.')[0]
        try:
            index = int(index_str)
            if index > max_index:
                max_index = index
                max_file = file_name
        except ValueError:
            pass

if max_file:
    print(f"Файл с максимальным индексом: {max_file}")
else:
    print("Файлы не найдены.")

##### Time Dependent

In [ ]:
def create_model_TimeDependent(
    path2testCase:str=os.path.join(os.getcwd(),'NACA2412','0','case.csv'),
    N_case:int=50000,
    path2foil:str=os.path.join(os.getcwd(),'NACA2412','0','foil.csv'),
    N_foil:int=40000,
    initConfig:str=os.path.join(os.getcwd(),'NACA0012','0','caseInfo.json')
):

    # Определение граничных областей
    def boundary_farfield_inlet(x, on_boundary):        return on_boundary and np.isclose(x[0], xmin)
    def boundary_farfield_top_bottom(x, on_boundary):        return on_boundary and (np.isclose(x[1], ymax) or np.isclose(x[1], ymin))
    def boundary_farfield_outlet(x, on_boundary):        return on_boundary and np.isclose(x[0], xmax)
    def boundary_airfoil(x, on_boundary):
        if x.shape == (3,) :return on_boundary and point_on_contour(contour_points=foil_points,point=x[:-1],threshold=1e-5)
        
    def point_on_contour(contour_points, point, threshold=1e-6):
        distances = cdist([point], contour_points)
        return np.any(np.abs(distances) < threshold)
        
    df = pd.read_csv(path2testCase).groupby('Time').apply(lambda x: x.sample(min(N_case, len(x)))).reset_index(drop=True)
    foil_points = pd.read_csv(path2foil).sample(n=N_foil)
    with open(initConfig, 'r') as file:
        # Load the JSON data from the file into a Python object
        infoCase = json.load(file)

    anchors = df[[ 'x', 'y','Time',]].values
    
    print('1')
    
    xmin, ymin, xmax, ymax, UMax, VMax, pMax = [infoCase[name] for name in ['xmin', 'ymin', 'xmax', 'ymax', 'UMax', 'VMax', 'pMax']]
    print(f'xmin, ymin, xmax, ymax, UMax, VMax, pMax: {xmin, ymin, xmax, ymax, UMax, VMax, pMax}')
    
    timeSteps = df['Time'].values.reshape(-1, 1)
    print(f'timeSteps: {timeSteps}')
    xmin_rec, xmax_rec = xmin + (-xmin + xmax)*1/4, xmin + (-xmin + xmax)*3/4
    ymin_rec, ymax_rec = ymin + (-ymin + ymax)*1/4, ymin + (-ymin + ymax)*3/4
    
    # Inittial
    dde.config.set_random_seed(48)
    dde.config.set_default_float('float32')


    farfield    = dde.geometry.Rectangle([xmin, ymin], [xmax, ymax])
    airfoil     = Polygon_v2(foil_points)
    geom        = dde.geometry.CSGDifference(farfield, airfoil)
    timedomain  = dde.geometry.TimeDomain(np.min(timeSteps), np.max(timeSteps))
    
    inner_rec   = dde.geometry.Rectangle([xmin_rec, ymin_rec], [xmax_rec, ymax_rec])
    outer_dom   = dde.geometry.CSGDifference(farfield, inner_rec)
    outer_dom   = dde.geometry.CSGDifference(outer_dom, airfoil)
    inner_dom   = dde.geometry.CSGDifference(inner_rec, airfoil)
    print('2')
    
    def distance2boundary(self,x):
        return self.geometry.distance2boundary(x[:,:-1])
    
    airfoil_XTime   = dde.geometry.GeometryXTime(airfoil, timedomain)
    airfoil_XTime.distance2boundary = types.MethodType(distance2boundary, airfoil_XTime)
     
    # Извлечение массивов U, V, p в массивы U_true, V_true, p_true 
    geomtime = dde.geometry.GeometryXTime(geom, timedomain)
    print('3')

    del inner_rec
    del outer_dom
    del inner_dom 

    def fun_U_farfield(x, y, _):return y[:,0:1] - UMax
    def fun_no_slip_U(x, y, _): return y[:,0:1]
    def fun_V_farfield(x, y, _): return  y[:,1:2] - 0
    def fun_no_slip_V(x, y, _): return y[:,1:2]
    
    ## U # м^(1)*с^(-2)
    bc_inlet_u = dde.icbc.OperatorBC(geomtime, fun_no_slip_U, boundary_farfield_inlet)
    bc_outlet_u = dde.icbc.OperatorBC(geomtime, fun_U_farfield, boundary_farfield_outlet)
    bc_top_bottom_u = dde.icbc.OperatorBC(geomtime, fun_no_slip_U, boundary_farfield_top_bottom)
    bc_airfoil_u = dde.icbc.OperatorBC(geomtime, fun_no_slip_U, boundary_airfoil)
    
    bc_U = [bc_inlet_u, bc_outlet_u, bc_top_bottom_u, bc_airfoil_u]
    observe_U = dde.icbc.PointSetBC(anchors, df['U'].values, component=0)
    print('4')
    
    ## V # м^(1)*с^(-2)
    bc_airfoil_v = dde.OperatorBC(geomtime, fun_no_slip_V, boundary_airfoil)
    bc_inlet_v = dde.OperatorBC(geomtime, fun_no_slip_V, boundary_farfield_inlet)
    bc_outlet_v = dde.OperatorBC(geomtime,fun_V_farfield, boundary_farfield_outlet)
    bc_top_bottom_v = dde.OperatorBC(geomtime, fun_no_slip_V, boundary_farfield_top_bottom)
    
    bc_V = [bc_inlet_v,bc_outlet_v,bc_top_bottom_v,bc_airfoil_v]
    observe_V = dde.icbc.PointSetBC(anchors, df['V'].values, component=1)
    print('5')

    pout = 0 # м^(2)*с^(-2)
    internalField_p_value = pout
    
    def internalField_p(x,y,_):
      p         = y[:, 2:3] # Давление
      if dde.backend.backend_name=='tensorflow.compat.v1':
        return p  - tf.cast(internalField_p_value, tf.float32)
      elif dde.backend.backend_name=='pytorch':
        return p  - torch.tensor(internalField_p_value, dtype=torch.float32)
    
    def zeroGradient_p(x,y,_):
      p = y[:, 2:3]
      return dde.grad.jacobian(p, x, j  = 0) + dde.grad.jacobian(p, x,  j  = 1)
    
    bc_inlet_p = dde.icbc.OperatorBC(geomtime, internalField_p, boundary_farfield_inlet,)
    bc_outlet_p = dde.icbc.OperatorBC(geomtime, internalField_p, boundary_farfield_outlet)
    bc_top_bottom_p = dde.icbc.OperatorBC(geomtime, internalField_p, boundary_farfield_top_bottom)
    bc_airfoil_p = dde.icbc.OperatorBC(geomtime, zeroGradient_p, boundary_airfoil)
    bc_p = [bc_inlet_p,bc_outlet_p,bc_top_bottom_p,bc_airfoil_p]
    observe_p = dde.icbc.PointSetBC(anchors, df['p'].values, component=2)
    print('6')

    data = dde.data.TimePDE(
        geomtime,
        navier_stokes,
        [
            *bc_U,        observe_U, # 5
            *bc_V,        observe_V, # 10
            *bc_p,        observe_p, # 35
            ],
        num_domain = 0, num_boundary = 0,
        num_test = 5000,
        anchors = anchors,
    )
    print('7')
    # FNN
    net = dde.nn.FNN([3] + [50] * 10 + [6], 'tanh', 'Glorot uniform')
    
    # Model definition
    model = dde.Model(data, net)
    model.compile(optimizer = 'adam', lr = 5e-4)
    return model, df, foil_points

def getU(x, y):    return y[:, 0:1]
def getV(x, y):    return y[:, 1:2]
def getP(x, y):    return y[:, 2:3]


In [ ]:
def create_model_TimeDependent(
    path2testCase:str=os.path.join(os.getcwd(),'NACA2412','0','case.csv'),
    N_case:int=50000,
    path2foil:str=os.path.join(os.getcwd(),'NACA2412','0','foil.csv'),
    N_foil:int=40000,
    initConfig:str=os.path.join(os.getcwd(),'NACA0012','0','caseInfo.json')
):

    # Определение граничных областей
    def boundary_farfield_inlet(x, on_boundary):        return on_boundary and np.isclose(x[0], xmin)
    def boundary_farfield_top_bottom(x, on_boundary):        return on_boundary and (np.isclose(x[1], ymax) or np.isclose(x[1], ymin))
    def boundary_farfield_outlet(x, on_boundary):        return on_boundary and np.isclose(x[0], xmax)
    def boundary_airfoil(x, on_boundary):
        if x.shape == (3,) :return on_boundary and point_on_contour(contour_points=foil_points,point=x[:-1],threshold=1e-5)
        
    def point_on_contour(contour_points, point, threshold=1e-6):
        distances = cdist([point], contour_points)
        return np.any(np.abs(distances) < threshold)
        
    df = pd.read_csv(path2testCase).groupby('Time').apply(lambda x: x.sample(min(N_case, len(x)))).reset_index(drop=True)
    foil_points = pd.read_csv(path2foil).sample(n=N_foil)
    with open(initConfig, 'r') as file:
        # Load the JSON data from the file into a Python object
        infoCase = json.load(file)

    anchors = df[[ 'x', 'y','Time',]].values
    
    print('1')
    
    xmin, ymin, xmax, ymax, UMax, VMax, pMax = [infoCase[name] for name in ['xmin', 'ymin', 'xmax', 'ymax', 'UMax', 'VMax', 'pMax']]
    print(f'xmin, ymin, xmax, ymax, UMax, VMax, pMax: {xmin, ymin, xmax, ymax, UMax, VMax, pMax}')
    
    timeSteps = df['Time'].values.reshape(-1, 1)
    print(f'timeSteps: {timeSteps}')
    xmin_rec, xmax_rec = xmin + (-xmin + xmax)*1/4, xmin + (-xmin + xmax)*3/4
    ymin_rec, ymax_rec = ymin + (-ymin + ymax)*1/4, ymin + (-ymin + ymax)*3/4
    
    # Inittial
    dde.config.set_random_seed(48)
    dde.config.set_default_float('float32')


    farfield    = dde.geometry.Rectangle([xmin, ymin], [xmax, ymax])
    airfoil     = Polygon_v2(foil_points)
    geom        = dde.geometry.CSGDifference(farfield, airfoil)
    timedomain  = dde.geometry.TimeDomain(np.min(timeSteps), np.max(timeSteps))
    
    inner_rec   = dde.geometry.Rectangle([xmin_rec, ymin_rec], [xmax_rec, ymax_rec])
    outer_dom   = dde.geometry.CSGDifference(farfield, inner_rec)
    outer_dom   = dde.geometry.CSGDifference(outer_dom, airfoil)
    inner_dom   = dde.geometry.CSGDifference(inner_rec, airfoil)
    print('2')
    
    def distance2boundary(self,x):
        return self.geometry.distance2boundary(x[:,:-1])
    
    airfoil_XTime   = dde.geometry.GeometryXTime(airfoil, timedomain)
    airfoil_XTime.distance2boundary = types.MethodType(distance2boundary, airfoil_XTime)
     
    # Извлечение массивов U, V, p в массивы U_true, V_true, p_true 
    geomtime = dde.geometry.GeometryXTime(geom, timedomain)
    print('3')

    del inner_rec
    del outer_dom
    del inner_dom 

    def fun_U_farfield(x, y, _):return y[:,0:1] - UMax
    def fun_no_slip_U(x, y, _): return y[:,0:1]
    def fun_V_farfield(x, y, _): return  y[:,1:2] - 0
    def fun_no_slip_V(x, y, _): return y[:,1:2]
    
    ## U # м^(1)*с^(-2)
    bc_inlet_u = dde.icbc.OperatorBC(geomtime, fun_no_slip_U, boundary_farfield_inlet)
    bc_outlet_u = dde.icbc.OperatorBC(geomtime, fun_U_farfield, boundary_farfield_outlet)
    bc_top_bottom_u = dde.icbc.OperatorBC(geomtime, fun_no_slip_U, boundary_farfield_top_bottom)
    bc_airfoil_u = dde.icbc.OperatorBC(geomtime, fun_no_slip_U, boundary_airfoil)
    
    bc_U = [bc_inlet_u, bc_outlet_u, bc_top_bottom_u, bc_airfoil_u]
    observe_U = dde.icbc.PointSetBC(anchors, df['U'].values, component=0)
    print('4')
    
    ## V # м^(1)*с^(-2)
    bc_airfoil_v = dde.OperatorBC(geomtime, fun_no_slip_V, boundary_airfoil)
    bc_inlet_v = dde.OperatorBC(geomtime, fun_no_slip_V, boundary_farfield_inlet)
    bc_outlet_v = dde.OperatorBC(geomtime,fun_V_farfield, boundary_farfield_outlet)
    bc_top_bottom_v = dde.OperatorBC(geomtime, fun_no_slip_V, boundary_farfield_top_bottom)
    
    bc_V = [bc_inlet_v,bc_outlet_v,bc_top_bottom_v,bc_airfoil_v]
    observe_V = dde.icbc.PointSetBC(anchors, df['V'].values, component=1)
    print('5')

    pout = 0 # м^(2)*с^(-2)
    internalField_p_value = pout
    
    def internalField_p(x,y,_):
      p         = y[:, 2:3] # Давление
      if dde.backend.backend_name=='tensorflow.compat.v1':
        return p  - tf.cast(internalField_p_value, tf.float32)
      elif dde.backend.backend_name=='pytorch':
        return p  - torch.tensor(internalField_p_value, dtype=torch.float32)
    
    def zeroGradient_p(x,y,_):
      p = y[:, 2:3]
      return dde.grad.jacobian(p, x, j  = 0) + dde.grad.jacobian(p, x,  j  = 1)
    
    bc_inlet_p = dde.icbc.OperatorBC(geomtime, internalField_p, boundary_farfield_inlet,)
    bc_outlet_p = dde.icbc.OperatorBC(geomtime, internalField_p, boundary_farfield_outlet)
    bc_top_bottom_p = dde.icbc.OperatorBC(geomtime, internalField_p, boundary_farfield_top_bottom)
    bc_airfoil_p = dde.icbc.OperatorBC(geomtime, zeroGradient_p, boundary_airfoil)
    bc_p = [bc_inlet_p,bc_outlet_p,bc_top_bottom_p,bc_airfoil_p]
    observe_p = dde.icbc.PointSetBC(anchors, df['p'].values, component=2)
    print('6')

    data = dde.data.TimePDE(
        geomtime,
        navier_stokes,
        [
            *bc_U,        observe_U, # 5
            *bc_V,        observe_V, # 10
            *bc_p,        observe_p, # 35
            ],
        num_domain = 0, num_boundary = 0,
        num_test = 5000,
        anchors = anchors,
    )
    print('7')
    # FNN
    net = dde.nn.FNN([3] + [50] * 10 + [6], 'tanh', 'Glorot uniform')
    
    # Model definition
    model = dde.Model(data, net)
    model.compile(optimizer = 'adam', lr = 5e-4)
    return model, df, foil_points

def getU(x, y):    return y[:, 0:1]
def getV(x, y):    return y[:, 1:2]
def getP(x, y):    return y[:, 2:3]




0

In [ ]:
model, df, foil_cont = create_model_TimeDependent(
    path2testCase=os.path.join(os.getcwd(),'NACA2412','0','case.csv'),
    N_case=30000,
    path2foil=os.path.join(os.getcwd(),'NACA2412','0','foil.csv'),
    N_foil=10000,
    initConfig=os.path.join(os.getcwd(),'NACA0012','caseInfo.json')
)
# To restore a model, use Model.restore.
model.restore(
    os.path.join(
            os.getcwd(),
            'model',
            'model.ckpt-300000.pt')
    )  # Replace ? with the exact filename
plt.figure(figsize = (32, 18))
plt.scatter(model.data.train_x_all[:,0], model.data.train_x_all[:,1], s = 0.3)
plt.axis('equal')

In [ ]:
# Извлечение значений колонок Time, x, y в массив points
X = df[['x', 'y','Time',]].values
df['U_pred'] = model.predict(X, operator = getU)
df['V_pred'] = model.predict(X, operator = getV)
df['p_pred'] = model.predict(X, operator = getP)

# df.to_csv('result.csv.gzip',compression='gzip')
images = []

for time in df['Time'].unique():
    first_time_data = df[df['Time'] == time]
    x, y = first_time_data['x'], first_time_data['y']
    
    U_true, V_true, p_true = first_time_data['U'], first_time_data['V'], first_time_data['p']
    U_pred, V_pred, p_pred = first_time_data['U_pred'], first_time_data['V_pred'], first_time_data['p_pred']
    
    U_diff, V_diff, p_diff = U_true - U_pred, V_true - V_pred, p_true - p_pred

    fig, axs = plt.subplots(3, 3, figsize=(24, 24))

    for i, (field_true, field_pred, field_diff, title,um) in enumerate(zip([U_true, V_true, p_true], [U_pred, V_pred, p_pred], [U_diff, V_diff, p_diff], ['U', 'V', 'p'],['m/s','m/s','m^2/s^2',])):
        contour_true = axs[i, 0].tricontourf(x, y, field_true, levels=100, cmap='turbo')
        axs[i, 0].set_xlabel('X')
        axs[i, 0].set_ylabel('Y')
        axs[i, 0].set_title(f'Field {title}_true {um}')
        fig.colorbar(contour_true, ax=axs[i, 0], label=title)

        contour_pred = axs[i, 1].tricontourf(x, y, field_pred, levels=100, cmap='turbo')
        axs[i, 1].set_xlabel('X')
        axs[i, 1].set_ylabel('Y')
        axs[i, 1].set_title(f'Field {title}_pred {um}')
        fig.colorbar(contour_pred, ax=axs[i, 1], label=title)

        contour_diff = axs[i, 2].tricontourf(x, y, field_diff, levels=100, cmap='turbo')
        axs[i, 2].set_xlabel('X')
        axs[i, 2].set_ylabel('Y')
        axs[i, 2].set_title(f'Field {title}_diff {um}')
        fig.colorbar(contour_diff, ax=axs[i, 2], label=title)

        for ax in axs[i]:
            patch = matplotlib.patches.Polygon(foil_cont[['x', 'y']], closed=True, fill=True, edgecolor='none', facecolor='black')
            ax.add_patch(patch)
     
    fig.savefig(f"subplot_{time}.png", bbox_inches='tight', dpi=300)
    images.append(Image.open(f"subplot_{time}.png"))
    plt.close(fig)

images[0].save('subplots.gif', save_all=True, append_images=images[1:], optimize=False, duration=500, loop=0)

for time in df['Time'].unique():
    os.remove(f"subplot_{time}.png")

### Incompressible

In [ ]:
def navier_stokes_incomp(x, y):
    """
    System of PDEs to be minimized: incompressible Navier-Stokes equation in the
    continuum-mechanics based formulations.
    """
    u         = y[:, 0:1] # Функция тока
    v         = y[:, 1:2] # Функция тока
    p         = y[:, 2:3] # Давление
    # Производные компонент скорости
    u_x = dde.grad.jacobian(u, x, i = 0, j = 0)
    u_y = dde.grad.jacobian(u, x, i = 0, j = 1)
    v_x = dde.grad.jacobian(v, x, i = 0, j = 0)
    v_y = dde.grad.jacobian(v, x, i = 0, j = 1)
    u_xx = dde.grad.hessian(u, x, i = 0, j = 0)
    u_yy = dde.grad.hessian(u, x, i = 1, j = 1)
    v_xx = dde.grad.hessian(v, x, i = 0, j = 0)
    v_yy = dde.grad.hessian(v, x, i = 1, j = 1)
    # Производные давления
    p_x = dde.grad.jacobian(p, x, j = 0)
    p_y = dde.grad.jacobian(p, x, j = 1)
    # Уравненияя сохранения массы
    eq_1 = u_x + v_y
    #  Уравнение движения (уравнение Навье-Стокса в компоненте x)
    eq_2_1 = rho * (u * u_x + v * u_y) - (
        - p_x + mu*(u_xx + u_yy) 
        )
    #  Уравнение движения (уравнение Навье-Стокса в компоненте y)
    eq_2_2 = rho * (u * v_x + v * v_y) - (
        - p_y + mu*(v_xx + v_yy) 
        )

    return [eq_1, eq_2_1, eq_2_2]

#### Train

In [ ]:
# From file
numInternal         = 10000
numXmax             = 2500
numXmin             = 2500
numYmax             = 2500
numYmin             = 2500
# Make domain
inner_points_N      = 10000
outer_points_N      = 10000
farfield_points_N   = 2500
airfoil_points_N    = 2500

In [ ]:
# Чтение данных из CSV файла
df = pd.read_csv(os.path.join(os.getcwd(),'NACA0012', 'case.csv'))
# Open the JSON file in read mode
with open(os.path.join(os.getcwd(),'NACA0012', "caseInfo.json"), 'r') as file:
    # Load the JSON data from the file into a Python object
    infoCase = json.load(file)

xmin, ymin, xmax, ymax, UMax, VMax, pMax = [infoCase[name] for name in ['xmin', 'ymin', 'xmax', 'ymax', 'UMax', 'VMax', 'pMax']]
# Извлечение значений из DataFrame
U_selected = df['U'].values.reshape(-1, 1)
V_selected = df['V'].values.reshape(-1, 1)
P_selected = df['p'].values.reshape(-1, 1)
x_selected = df['x'].values.reshape(-1, 1)
y_selected = df['y'].values.reshape(-1, 1) 

xmin,xmax = x_selected.min(),x_selected.max(),
ymin,ymax = y_selected.min(),y_selected.max(),

In [ ]:
dde.config.set_random_seed(48)
dde.config.set_default_float('float32')

nacaParams = {
    'naca_code' : '0012',
    'c'         : 1,
    'offset_x'  : 0.0,
    'offset_y'  : 0.0,
    'angle_deg' : 0
}

from utils import *
foil_points = boundaryNACA(**nacaParams)

xmin_rec, xmax_rec = xmin + (-xmin + xmax)*1/4, xmin + (-xmin + xmax)*3/4
ymin_rec, ymax_rec = ymin + (-ymin + ymax)*1/4, ymin + (-ymin + ymax)*3/4

umax = UMax
vmax = VMax
global Re;  Re = rho * umax * nacaParams['c'] * 1 / mu

In [ ]:
farfield    = dde.geometry.Rectangle([xmin, ymin], [xmax, ymax])
airfoil     =   dde.geometry.Polygon(foil_points)

geom        = dde.geometry.CSGDifference(farfield, airfoil) 

inner_rec  = dde.geometry.Rectangle([xmin_rec, ymin_rec], [xmax_rec, ymax_rec])
outer_dom  = dde.geometry.CSGDifference(farfield, inner_rec)
outer_dom  = dde.geometry.CSGDifference(outer_dom, airfoil)
inner_dom  = dde.geometry.CSGDifference(inner_rec, airfoil)

inner_points = inner_dom.random_points(inner_points_N)
outer_points = outer_dom.random_points(outer_points_N)
airfoil_points = airfoil.random_points(airfoil_points_N)

farfield_points = farfield.random_boundary_points(farfield_points_N)

ob_xy = np.hstack((x_selected, y_selected))

points = np.concatenate((
    inner_points,
    outer_points,
    farfield_points,
    foil_points,
    ob_xy
    ), axis=0)
 

In [ ]:
def boundary_farfield_inlet(x, on_boundary):    
    return on_boundary and np.isclose(x[0], xmin)
def boundary_farfield_top_bottom(x, on_boundary):    
    return on_boundary and (np.isclose(x[1], ymax) or np.isclose(x[1], ymin))
def boundary_farfield_outlet(x, on_boundary):   
    return on_boundary and np.isclose(x[0], xmax)
def boundary_airfoil(x, on_boundary): 
    return on_boundary and airfoil.on_boundary(np.array([x]))[0]
# Boundary values definition
def fun_u_farfield(x, y, _):   
    u         = y[:, 0:1] # Функция тока
    return u - umax
    
def fun_v_farfield(x, y, _):
    v         = y[:, 1:2] 
    return v - vmax
# foil
def fun_no_slip_u(x, y, _): 
    u         = y[:, 0:1] # Функция тока 
    return u  
def fun_no_slip_v(x, y, _): 
    v         = y[:, 1:2] # Функция тока   
    return v  
# outlet
def fun_outlet_u(x, y, _): 
    u = y[:, 0:1] # Функция тока 
    u_x = dde.grad.jacobian(u,x,j  = 0)      
    return u_x  
def fun_outlet_v(x, y, _): 
    v         = y[:, 1:2] # Функция тока 
    v_x = dde.grad.jacobian(v,x,j  = 0)   
    return v_x  
# top bottom
def fun_top_bottom_u(x, y, _): 
    u = y[:, 0:1] # Функция тока 
    u_y = dde.grad.jacobian(u,x,j  = 1)      
    return u_y  
def fun_top_bottom_v(x, y, _): 
    v         = y[:, 1:2] # Функция тока 
    v_y = dde.grad.jacobian(v,x,j  = 1)   
    return v_y  

In [ ]:
from tqdm import tqdm

# Счетчики для каждой граничной области
count_farfield_inlet = 0
count_farfield_top_bottom = 0
count_farfield_outlet = 0
count_airfoil = 0

# Перебираем каждую точку в массиве points с прогресс-баром
for point in tqdm(points, desc="Checking boundary points"):
    x = point[0]  # Координата x точки
    y = point[1]  # Координата y точки
    
    # Проверяем, принадлежит ли точка к граничной области входа
    if boundary_farfield_inlet([x, y], on_boundary=True):
        count_farfield_inlet += 1
    
    # Проверяем, принадлежит ли точка к граничной области верха или низа
    if boundary_farfield_top_bottom([x, y], on_boundary=True):
        count_farfield_top_bottom += 1
    
    # Проверяем, принадлежит ли точка к граничной области выхода
    if boundary_farfield_outlet([x, y], on_boundary=True):
        count_farfield_outlet += 1
    
    # Проверяем, принадлежит ли точка к граничной области профиля
    if boundary_airfoil([x, y], on_boundary=True):
        count_airfoil += 1

# Выводим количество точек на каждой граничной области
print("Количество точек на граничной области входа:", count_farfield_inlet)
print("Количество точек на граничной области верха или низа:", count_farfield_top_bottom)
print("Количество точек на граничной области выхода:", count_farfield_outlet)
print("Количество точек на граничной области профиля:", count_airfoil)

In [ ]:
## U # м^(1)*с^(-2)
bc_inlet_u = dde.icbc.OperatorBC(geom, fun_u_farfield, boundary_farfield_inlet)
bc_top_bottom_u = dde.icbc.OperatorBC(geom, fun_top_bottom_u, boundary_farfield_top_bottom)
bc_outlet_u = dde.OperatorBC(geom,fun_outlet_u, boundary_farfield_outlet)
bc_airfoil_u = dde.icbc.OperatorBC(geom, fun_no_slip_u, boundary_airfoil)

bc_U = [bc_inlet_u,bc_outlet_u, bc_top_bottom_u, bc_airfoil_u]
observe_U = dde.icbc.PointSetBC(ob_xy, U_selected, component=0)

In [ ]:
## V # м^(1)*с^(-2)
bc_inlet_v = dde.OperatorBC(geom, fun_v_farfield, boundary_farfield_inlet)
bc_top_bottom_v = dde.OperatorBC(geom, fun_top_bottom_v, boundary_farfield_top_bottom)
bc_outlet_v = dde.OperatorBC(geom,fun_outlet_v, boundary_farfield_outlet)
bc_airfoil_v = dde.OperatorBC(geom, fun_no_slip_v, boundary_airfoil) 

bc_V = [bc_inlet_v,bc_outlet_v ,bc_top_bottom_v,bc_airfoil_v]
observe_V = dde.icbc.PointSetBC(ob_xy, V_selected, component=1)

In [ ]:
pout = 0 # м^(2)*с^(-2)
internalField_p_value = pout
# inlet farfield
def internalField_p(x,y,_):
  p = y[:, 2:3] # Давление 
  if dde.backend.backend_name=='tensorflow.compat.v1':
    return p - tf.cast(internalField_p_value, tf.float32)
  elif dde.backend.backend_name=='pytorch':
    return p - torch.tensor(internalField_p_value, dtype=torch.float32)
# foil
def zeroGradient_p(x,y,_):
  p = y[:, 2:3] # Давление 
  p_x = dde.grad.jacobian(p, x, j  = 0)
  p_y = dde.grad.jacobian(p, x,  j  = 1)
  return p_x + p_y
#  outlet
def outlet_p(x,y,_):
  p = y[:, 2:3] # Давление 
  p_x = dde.grad.jacobian(p, x, j  = 0)
  return p_x
# top bottom 
def top_bottom_p(x,y,_):
  p = y[:, 2:3] # Давление 
  p_y = dde.grad.jacobian(p, x,  j  = 1)
  return  p_y


bc_inlet_p = dde.icbc.OperatorBC(
    geom, internalField_p, boundary_farfield_inlet, #component=5
    )
bc_outlet_p = dde.icbc.OperatorBC(
    geom, outlet_p, boundary_farfield_outlet, #component=5
    )
bc_top_bottom_p = dde.icbc.OperatorBC(
    geom, top_bottom_p, boundary_farfield_top_bottom, #component=5
    )
bc_airfoil_p = dde.icbc.OperatorBC(
    geom, zeroGradient_p, boundary_airfoil,#component = 5
    )

bc_p = [bc_inlet_p,bc_outlet_p,bc_top_bottom_p,bc_airfoil_p]

observe_p = dde.icbc.PointSetBC(ob_xy, P_selected, component=2)

In [ ]:
# Problem setup
data = dde.data.PDE(
    geom,
    navier_stokes_incomp,
    [
        *bc_U,        observe_U,  
        *bc_V,        observe_V,  
        *bc_p,        observe_p,  
        ],
    num_domain = 3000, num_boundary = 2000,
    num_test = 5000,
    anchors = points,
)
plt.figure(figsize = (32, 18))
plt.scatter(data.train_x_all[:,0], data.train_x_all[:,1], s = 0.3)
plt.scatter(ob_xy[:,0],ob_xy[:,1], s = 0.75, color='red')
plt.axis('equal')

##### FNN

In [ ]:
# Neural network definition
import tensorflow as tf

layer_size  = [2] + [50] * 10 + [3]
activation  = 'tanh'
initializer = 'Glorot uniform'

# FNN
net = dde.nn.FNN(layer_size, activation, initializer)

model_path = os.path.join(os.getcwd(),'model')
if not os.path.exists(model_path):
    os.makedirs(model_path)
    print(f"Папка '{model_path}' создана успешно.")
else:
    print(f"Папка '{model_path}' уже существует.")


# Model definition
model = dde.Model(data, net)
model.compile(optimizer = 'adam', lr = 5e-4)

checker = dde.callbacks.ModelCheckpoint(
    os.path.join(model_path,'model.ckpt'),
    save_better_only=True,
    period=10000
)


train_story_path = os.path.join(os.getcwd(),'train_story')
if not os.path.exists(train_story_path):
    os.makedirs(train_story_path)
    print(f"Папка '{train_story_path}' создана успешно.")
else:
    print(f"Папка '{train_story_path}' уже существует.")

losshistory, train_state = model.train(
    epochs = 20_000, display_every = 250,
    model_save_path = os.path.join(train_story_path,'train_story'),
    callbacks=[checker]
    )
dde.saveplot(losshistory, train_state, issave = True, isplot = True)

##### PFNN

In [ ]:
# Neural network definition
layer_size = [2, [35, 35, 35], [35, 35, 35],[35, 35, 35], 3]
activation  = 'tanh'
initializer = 'Glorot uniform'

# FNN
net = dde.nn.PFNN(layer_size, activation, initializer)

model_path = os.path.join(os.getcwd(),'model_PFNN')
if not os.path.exists(model_path):
    os.makedirs(model_path)
    print(f"Папка '{model_path}' создана успешно.")
else:
    print(f"Папка '{model_path}' уже существует.")


# Model definition
model = dde.Model(data, net)
model.compile(optimizer = 'adam', lr = 5e-4)

checker = dde.callbacks.ModelCheckpoint(
    os.path.join(model_path,'model.ckpt'),
    save_better_only=True,
    period=10000
)


train_story_path = os.path.join(os.getcwd(),'train_story_PFNN')
if not os.path.exists(train_story_path):
    os.makedirs(train_story_path)
    print(f"Папка '{train_story_path}' создана успешно.")
else:
    print(f"Папка '{train_story_path}' уже существует.")

losshistory, train_state = model.train(
    epochs = 20000, display_every = 250,
    model_save_path = os.path.join(train_story_path,'train_story_PFNN'),
    callbacks=[checker]
    )
dde.saveplot(losshistory, train_state, issave = True, isplot = True)

#### Test

In [ ]:
def create_model_TimeIndependent(
    path2testCase:str=os.path.join(os.getcwd(),'NACA2412','0','case.csv'),
    N_case:int=50000,
    path2foil:str=os.path.join(os.getcwd(),'NACA2412','0','foil.csv'),
    N_foil:int=40000,
    initConfig:str=os.path.join(os.getcwd(),'NACA0012','0','caseInfo.json')
):
    def navier_stokes_incomp(x, y):
        """
        System of PDEs to be minimized: incompressible Navier-Stokes equation in the
        continuum-mechanics based formulations.
        """
        u         = y[:, 0:1] # Функция тока
        v         = y[:, 1:2] # Функция тока
        p         = y[:, 2:3] # Давление
        # Производные компонент скорости
        u_x = dde.grad.jacobian(u, x, i = 0, j = 0)
        u_y = dde.grad.jacobian(u, x, i = 0, j = 1)
        v_x = dde.grad.jacobian(v, x, i = 0, j = 0)
        v_y = dde.grad.jacobian(v, x, i = 0, j = 1)
        u_xx = dde.grad.hessian(u, x, i = 0, j = 0)
        u_yy = dde.grad.hessian(u, x, i = 1, j = 1)
        v_xx = dde.grad.hessian(v, x, i = 0, j = 0)
        v_yy = dde.grad.hessian(v, x, i = 1, j = 1)
        # Производные давления
        p_x = dde.grad.jacobian(p, x, j = 0)
        p_y = dde.grad.jacobian(p, x, j = 1)
        # Уравненияя сохранения массы
        eq_1 = u_x + v_y
        #  Уравнение движения (уравнение Навье-Стокса в компоненте x)
        eq_2_1 = rho * (u * u_x + v * u_y) - (
            - p_x + mu*(u_xx + u_yy) 
            )
        #  Уравнение движения (уравнение Навье-Стокса в компоненте y)
        eq_2_2 = rho * (u * v_x + v * v_y) - (
            - p_y + mu*(v_xx + v_yy) 
            )
    
        return [eq_1, eq_2_1, eq_2_2]

    def sample_points_by_patch(df, N):
        patches = df['patch'].unique()
        points_per_patch = N // len(patches)
        extra_points = N % len(patches)
        
        sampled_points = []
        remaining_points = []

        for patch in patches:
            patch_df = df[df['patch'] == patch]
            if len(patch_df) <= points_per_patch:
                sampled_points.append(patch_df)
                extra_points += points_per_patch - len(patch_df)
            else:
                sampled_points.append(patch_df.sample(n=points_per_patch))
        
        remaining_points = df.drop(pd.concat(sampled_points).index)
        sampled_points.append(remaining_points.sample(n=extra_points))
        
        return pd.concat(sampled_points)

    # Чтение данных из CSV файла
    df = sample_points_by_patch(pd.read_csv(path2testCase), N_case)
    foil_points = pd.read_csv(path2foil).sample(n=N_foil)
    # Open the JSON file in read mode
    with open(initConfig, 'r') as file:
        # Load the JSON data from the file into a Python object
        infoCase = json.load(file)

    anchors = df[[ 'x', 'y',]].values
    
    xmin, ymin, xmax, ymax, UMax, VMax, pMax = [infoCase[name] for name in ['xmin', 'ymin', 'xmax', 'ymax', 'UMax', 'VMax', 'pMax']]
    # Извлечение значений из DataFrame
    U_selected = df['U'].values.reshape(-1, 1);    V_selected = df['V'].values.reshape(-1, 1)
    P_selected = df['p'].values.reshape(-1, 1)
    x_selected = df['x'].values.reshape(-1, 1);    y_selected = df['y'].values.reshape(-1, 1) 
    
    xmin,xmax = x_selected.min(),x_selected.max(),
    ymin,ymax = y_selected.min(),y_selected.max(),
    dde.config.set_random_seed(48)
    dde.config.set_default_float('float32')
    
    xmin_rec, xmax_rec = xmin + (-xmin + xmax)*1/4, xmin + (-xmin + xmax)*3/4
    ymin_rec, ymax_rec = ymin + (-ymin + ymax)*1/4, ymin + (-ymin + ymax)*3/4
    
    umax = UMax
    vmax = VMax

    farfield    = dde.geometry.Rectangle([xmin, ymin], [xmax, ymax])
    airfoil     =   dde.geometry.Polygon(foil_points)
    
    geom        = dde.geometry.CSGDifference(farfield, airfoil)  
    
    points = anchors

    def boundary_farfield_inlet(x, on_boundary):    return on_boundary and np.isclose(x[0], xmin)
    def boundary_farfield_top_bottom(x, on_boundary):    return on_boundary and (np.isclose(x[1], ymax) or np.isclose(x[1], ymin))
    def boundary_farfield_outlet(x, on_boundary):    return on_boundary and np.isclose(x[0], xmax)
    def boundary_airfoil(x, on_boundary): return on_boundary and airfoil.on_boundary(np.array([x]))[0]
    
    # Boundary values definition
    def fun_u_farfield(x, y, _):   
        u         = y[:, 0:1] # Функция тока
        return u - umax
        
    def fun_v_farfield(x, y, _):
        v         = y[:, 1:2] 
        return v - vmax
    # foil
    def fun_no_slip_u(x, y, _): 
        u         = y[:, 0:1] # Функция тока 
        return u  
    def fun_no_slip_v(x, y, _): 
        v         = y[:, 1:2] # Функция тока   
        return v  
    
    # outlet
    def fun_outlet_u(x, y, _): 
        u = y[:, 0:1] # Функция тока 
        u_x = dde.grad.jacobian(u,x,j  = 0)      
        return u_x  
    def fun_outlet_v(x, y, _): 
        v         = y[:, 1:2] # Функция тока 
        v_x = dde.grad.jacobian(v,x,j  = 0)   
        return v_x  
    # top bottom
    def fun_top_bottom_u(x, y, _): 
        u = y[:, 0:1] # Функция тока 
        u_y = dde.grad.jacobian(u,x,j  = 1)      
        return u_y  
    def fun_top_bottom_v(x, y, _): 
        v         = y[:, 1:2] # Функция тока 
        v_y = dde.grad.jacobian(v,x,j  = 1)   
        return v_y  
    ## U # м^(1)*с^(-2)
    bc_inlet_u = dde.icbc.OperatorBC(geom, fun_u_farfield, boundary_farfield_inlet)
    bc_top_bottom_u = dde.icbc.OperatorBC(geom, fun_top_bottom_u, boundary_farfield_top_bottom)
    bc_outlet_u = dde.OperatorBC(geom,fun_outlet_u, boundary_farfield_outlet)
    bc_airfoil_u = dde.icbc.OperatorBC(geom, fun_no_slip_u, boundary_airfoil)
    
    bc_U = [bc_inlet_u,bc_outlet_u, bc_top_bottom_u, bc_airfoil_u]
    observe_U = dde.icbc.PointSetBC(anchors, U_selected, component=0)

    ## V # м^(1)*с^(-2)
    bc_inlet_v = dde.OperatorBC(geom, fun_v_farfield, boundary_farfield_inlet)
    bc_top_bottom_v = dde.OperatorBC(geom, fun_top_bottom_v, boundary_farfield_top_bottom)
    bc_outlet_v = dde.OperatorBC(geom,fun_outlet_v, boundary_farfield_outlet)
    bc_airfoil_v = dde.OperatorBC(geom, fun_no_slip_v, boundary_airfoil) 
    
    bc_V = [bc_inlet_v,bc_outlet_v ,bc_top_bottom_v,bc_airfoil_v]
    observe_V = dde.icbc.PointSetBC(anchors, V_selected, component=1)

    pout = 0 # м^(2)*с^(-2)
    internalField_p_value = pout
    # inlet farfield
    def internalField_p(x,y,_):
      p = y[:, 2:3] # Давление 
      if dde.backend.backend_name=='tensorflow.compat.v1':
        return p - tf.cast(internalField_p_value, tf.float32)
      elif dde.backend.backend_name=='pytorch':
        return p - torch.tensor(internalField_p_value, dtype=torch.float32)
    # foil
    def zeroGradient_p(x,y,_):
      p = y[:, 2:3] # Давление 
      p_x = dde.grad.jacobian(p, x, j  = 0)
      p_y = dde.grad.jacobian(p, x,  j  = 1)
      return p_x + p_y
    #  outlet
    def outlet_p(x,y,_):
      p = y[:, 2:3] # Давление 
      p_x = dde.grad.jacobian(p, x, j  = 0)
      return p_x
    # top bottom 
    def top_bottom_p(x,y,_):
      p = y[:, 2:3] # Давление 
      p_y = dde.grad.jacobian(p, x,  j  = 1)
      return  p_y
    
    
    bc_inlet_p = dde.icbc.OperatorBC(
        geom, internalField_p, boundary_farfield_inlet, #component=5
        )
    bc_outlet_p = dde.icbc.OperatorBC(
        geom, outlet_p, boundary_farfield_outlet, #component=5
        )
    bc_top_bottom_p = dde.icbc.OperatorBC(
        geom, top_bottom_p, boundary_farfield_top_bottom, #component=5
        )
    bc_airfoil_p = dde.icbc.OperatorBC(
        geom, zeroGradient_p, boundary_airfoil,#component = 5
        )
    
    bc_p = [bc_inlet_p,bc_outlet_p,bc_top_bottom_p,bc_airfoil_p]
    
    observe_p = dde.icbc.PointSetBC(anchors, P_selected, component=2)

    # Problem setup
    data = dde.data.PDE(
        geom,
        navier_stokes_incomp,
        [
            *bc_U,        observe_U, # 5
            *bc_V,        observe_V, # 10
            *bc_p,        observe_p, # 35
            ],
        num_domain = 0, num_boundary = 0,
        num_test = 5000,
        anchors = points,
    ) 
    layer_size  = [2] + [50] * 10 + [3]
    activation  = 'tanh'
    initializer = 'Glorot uniform'
    
    # FNN
    net = dde.nn.FNN(layer_size, activation, initializer)    
    # Model definition
    model = dde.Model(data, net)
    model.compile(optimizer = 'adam', lr = 5e-4)
    return model, df, foil_points


def getU(x, y):    return y[:, 0:1]
def getV(x, y):    return y[:, 1:2]
def getP(x, y):    return y[:, 2:3]


In [ ]:
model, df, foil_cont = create_model_TimeIndependent(
    path2testCase=os.path.join(os.getcwd(),'NACA0012','test','case.csv'),
    N_case=5000,
    path2foil=os.path.join(os.getcwd(),'NACA0012','test','foil.csv'),
    N_foil=1000,
    initConfig=os.path.join(os.getcwd(),'NACA0012','caseInfo.json')
)
# To restore a model, use Model.restore.
model.restore(
    os.path.join(
            os.getcwd(),
            'train_story',
            'train_story-250000.pt'),
    device=torch.device('cpu')
    )  # Replace ? with the exact filename
plt.figure(figsize = (32, 18))
plt.scatter(model.data.train_x_all[:,0], model.data.train_x_all[:,1], s = 0.3)
plt.axis('equal')

In [ ]:
# Извлечение значений колонок Time, x, y в массив points
X = df[['x', 'y']].values
df['U_pred'] = model.predict(X, operator = getU)
df['V_pred'] = model.predict(X, operator = getV)
df['p_pred'] = model.predict(X, operator = getP)

# df.to_csv('result.csv.gzip',compression='gzip')
images = []

x, y = df['x'], df['y']

U_true, V_true, p_true = df['U'], df['V'], df['p']
U_pred, V_pred, p_pred = df['U_pred'], df['V_pred'], df['p_pred']

U_diff, V_diff, p_diff = U_true - U_pred, V_true - V_pred, p_true - p_pred

fig, axs = plt.subplots(3, 3, figsize=(24, 24))

for i, (field_true, field_pred, field_diff, title,um) in enumerate(zip([U_true, V_true, p_true], [U_pred, V_pred, p_pred], [U_diff, V_diff, p_diff], ['U', 'V', 'p'],['m/s','m/s','m^2/s^2',])):
    contour_true = axs[i, 0].tricontourf(x, y, field_true, levels=100, cmap='turbo')
    axs[i, 0].set_xlabel('X')
    axs[i, 0].set_ylabel('Y')
    axs[i, 0].set_title(f'Field {title}_true {um}')
    fig.colorbar(contour_true, ax=axs[i, 0], label=title)

    contour_pred = axs[i, 1].tricontourf(x, y, field_pred, levels=100, cmap='turbo')
    axs[i, 1].set_xlabel('X')
    axs[i, 1].set_ylabel('Y')
    axs[i, 1].set_title(f'Field {title}_pred {um}')
    fig.colorbar(contour_pred, ax=axs[i, 1], label=title)

    contour_diff = axs[i, 2].tricontourf(x, y, field_diff, levels=100, cmap='turbo')
    axs[i, 2].set_xlabel('X')
    axs[i, 2].set_ylabel('Y')
    axs[i, 2].set_title(f'Field {title}_diff {um}')
    fig.colorbar(contour_diff, ax=axs[i, 2], label=title)

    for ax in axs[i]:
        patch = matplotlib.patches.Polygon(foil_cont[['x', 'y']], closed=True, fill=True, edgecolor='none', facecolor='black')
        ax.add_patch(patch)
 
fig.savefig(f"subplot_0.png", bbox_inches='tight', dpi=300)
images.append(Image.open(f"subplot_{time}.png"))
plt.close(fig)